In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import mixed_precision
from tensorflow.keras.applications import VGG19
from tensorflow.keras import regularizers

In [3]:
mixed_precision.set_global_policy('mixed_float16')
print("⚡ Mixed precision enabled")

tf.keras.backend.clear_session()


INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3050 Laptop GPU, compute capability 8.6
⚡ Mixed precision enabled


In [4]:
train_dir = "data2/AffectNet_Classify/train"
val_dir   = "data2/AffectNet_Classify/valid"

batch_size = 64
img_size = (160, 160)

train_gen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    rotation_range=10,
    zoom_range=0.08,
    width_shift_range=0.05,
    height_shift_range=0.05
).flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical",
    color_mode="rgb"
)

val_gen = ImageDataGenerator(
    rescale=1./255
).flow_from_directory(
    val_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical",
    color_mode="rgb"
)

num_classes = len(train_gen.class_indices)
print(f"Number of classes = {num_classes}")

Found 17101 images belonging to 8 classes.
Found 5406 images belonging to 8 classes.
Number of classes = 8


In [5]:
train_dataset = tf.data.Dataset.from_generator(
    lambda: train_gen,
    output_signature=( 
        tf.TensorSpec(shape=(None, *img_size, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(None, num_classes), dtype=tf.float32),
    )
).prefetch(tf.data.AUTOTUNE)

val_dataset = tf.data.Dataset.from_generator(
    lambda: val_gen,
    output_signature=(
        tf.TensorSpec(shape=(None, *img_size, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(None, num_classes), dtype=tf.float32),
    )
).prefetch(tf.data.AUTOTUNE)

In [6]:
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', patience=2, factor=0.3, min_lr=1e-7)
]


In [7]:
base_model = VGG19(
    include_top=False,
    weights="imagenet",
    input_shape=(*img_size, 3)
)

for layer in base_model.layers:
    if hasattr(layer, "kernel_regularizer"):
        layer.kernel_regularizer = regularizers.l2(1e-4)

base_model.trainable = False  

x = base_model.output
x = layers.GlobalAveragePooling2D()(x)

x = layers.BatchNormalization()(x)
x = layers.Dropout(0.3)(x)

x = layers.Dense(
    256,
    activation="relu",
    kernel_regularizer=regularizers.l1_l2(l1=1e-4, l2=1e-4)
)(x)

x = layers.BatchNormalization()(x)
x = layers.Dropout(0.3)(x)

output = layers.Dense(
    num_classes,
    activation="softmax",
    dtype="float32",
    kernel_regularizer=regularizers.l1(1e-4)
)(x)

model = models.Model(inputs=base_model.input, outputs=output)


In [9]:
# Fine-tuning

base_model.trainable = True

for layer in base_model.layers[:12]:
    layer.trainable = False

optimizer = mixed_precision.LossScaleOptimizer(
    tf.keras.optimizers.Adam(learning_rate=1e-5)
)

model.compile(
    optimizer=optimizer,
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

steps_per_epoch = train_gen.samples // batch_size
validation_steps = val_gen.samples // batch_size


model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=6,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    callbacks=callbacks
)


Epoch 1/6


267/267 [==============================] - 72s 263ms/step - loss: 3.0644 - accuracy: 0.2281 - val_loss: 2.4211 - val_accuracy: 0.3216 - lr: 1.0000e-05
Epoch 2/6
267/267 [==============================] - 67s 250ms/step - loss: 2.6377 - accuracy: 0.3413 - val_loss: 2.2602 - val_accuracy: 0.4295 - lr: 1.0000e-05
Epoch 3/6
267/267 [==============================] - 65s 244ms/step - loss: 2.3807 - accuracy: 0.4100 - val_loss: 2.0356 - val_accuracy: 0.4769 - lr: 1.0000e-05
Epoch 4/6
267/267 [==============================] - 81s 304ms/step - loss: 2.2438 - accuracy: 0.4440 - val_loss: 2.0030 - val_accuracy: 0.5123 - lr: 1.0000e-05
Epoch 5/6
267/267 [==============================] - 88s 329ms/step - loss: 2.1221 - accuracy: 0.4805 - val_loss: 1.8728 - val_accuracy: 0.5512 - lr: 1.0000e-05
Epoch 6/6
267/267 [==============================] - 90s 338ms/step - loss: 2.0066 - accuracy: 0.5119 - val_loss: 1.8885 - val_accuracy: 0.5450 - lr: 1.0000e-05


In [10]:
loss, acc = model.evaluate(val_dataset, steps=validation_steps)
print(f"\n🏁 Final Validation Accuracy: {acc*100:.2f}%")

model.save("vgg19_emotion_classifier.h5")

84/84 [==============================] - 9s 104ms/step - loss: 1.8906 - accuracy: 0.5446

🏁 Final Validation Accuracy: 54.46%
